In [1]:
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

from itertools import product
import pandas as pd
import math

from vsim.utils import get_processed_metadata

In [2]:
# Load data from the Excel file
file_path = "../data/VOSimu-InputInformation.xlsx"

loc_df, vehicles_df, co_df = get_processed_metadata(meta_file_path=file_path)

In [3]:
# Helper function to calculate Manhattan distance
def manhattan_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

In [120]:
class VSDataCenter:
    def __init__(self, data_file):
        self._locations = {}
        self._vehicles = {}
        self._container_orders = {}
        self._distance_matrix = None

        self._prepare_data(data_file)
        self._create_distance_matrix()

    def _prepare_data(self, data_file):
        locations_df, vehicles_df, container_orders_df = get_processed_metadata(meta_file_path=data_file)
        # Keep track of container orders by adding a delivery status
        container_orders_df['delivered'] = False

        self._locations = locations_df.set_index("location_name").to_dict("index")
        self._vehicles = vehicles_df.set_index("id").to_dict("index")
        self._container_orders = container_orders_df.set_index("co_id").to_dict("index")

    def _create_distance_matrix(self):
        num_locations = len(self._locations)
        distance_matrix = np.zeros((num_locations, num_locations))
        locations = list(self._locations.values())
        for i, j in product(range(num_locations), range(num_locations)):
            x1, y1, x2, y2 = locations[i]['x'], locations[i]['y'], locations[j]['x'], locations[j]['y']
            distance_matrix[i, j] = manhattan_distance(x1, y1, x2, y2)

        self._distance_matrix = distance_matrix

    def get_distance(self, loc_1, loc_2):
        loc_1_idx = list(self.locations.keys()).index(loc_1)
        loc_2_idx = list(self.locations.keys()).index(loc_2)
        return self.distance_matrix[loc_1_idx][loc_2_idx]

    def toggle_order_status(self, order_id):
        self._container_orders[order_id]['delivered'] = not self._container_orders[order_id]['delivered']

    def update_vehicle_location(self, vehicle_id, location):
        self._vehicles[vehicle_id]['start_location'] = location

    def order_already_delivered(self, order_id):
        return self._container_orders[order_id]['delivered']

    def get_remaining_orders(self):
        remaining_orders = {}
        for order_id, order_data in self._container_orders.items():
            if not self.order_already_delivered(order_id):
                remaining_orders[order_id] = order_data
        return remaining_orders

    @property
    def locations(self):
        return self._locations.copy()

    @property
    def vehicles(self):
        return self._vehicles.copy()

    @property
    def container_orders(self):
        return self._container_orders.copy()

    @property
    def distance_matrix(self):
        return np.copy(self._distance_matrix)





In [166]:
class VSSolver:
    def __init__(self, data_center):
        self._data_center: VSDataCenter = data_center

        self._solver = None
        self._var_x = None

        self._opt_obj = None
        self._opt_x = None

        self._capacity_violation_factor = 1.0
        self._opt_results = []

    def _build_model(self):
        self._solver = pywraplp.Solver.CreateSolver('SCIP')

        self._create_variables()
        self._create_objective()
        self._create_constraints()

    def optimize(self):
        status = None
        self._capacity_violation_factor = 1.0

        while status != pywraplp.Solver.OPTIMAL:
            print("factor: ", self._capacity_violation_factor)
            self._build_model()
            self._solver.SetTimeLimit(20000)
            status = self._solver.Solve()
            self._capacity_violation_factor *= 2
        print(status)
        self._opt_obj = self._solver.Objective().Value()
        self._opt_x = []
        for (v, o) in self._var_x:
            if self._var_x[v, o].solution_value() == 1:
                self._opt_x.append((v, o))

        self._opt_results.append((self._opt_x.copy(), self._opt_obj))
        self._update_environment()

    def _update_environment(self):
        for (v, o) in self._opt_x:
            # Remove already handled orders
            self._data_center.toggle_order_status(o)

            # Update vehicle locations to the destination of previously assigned orders
            self._data_center.update_vehicle_location(v, self._data_center.container_orders[o]['dest'])

    def _create_variables(self):
        vehicles = self._data_center.vehicles
        orders = self._data_center.get_remaining_orders()

        self._var_x = {}
        for v in vehicles:
            for o in orders:
                self._var_x[v, o] = self._solver.IntVar(0, 1, f'x[{v},{o}]')

    def _create_objective(self):
        vehicles = self._data_center.vehicles
        orders = self._data_center.get_remaining_orders()

        obj_expr = []
        for v, v_data in vehicles.items():
            for o, o_data in orders.items():
                v_loc = v_data['start_location']
                o_origin = o_data['origin']
                o_dest = o_data['dest']

                v_to_origin = self._data_center.get_distance(v_loc, o_origin)
                origin_to_dest = self._data_center.get_distance(o_origin, o_dest)

                obj_expr.append((v_to_origin + origin_to_dest) * self._var_x[v, o])

        self._solver.Minimize(self._solver.Sum(obj_expr))

    def _create_constraints(self):
        locations = self._data_center.locations
        vehicles = self._data_center.vehicles
        orders = self._data_center.get_remaining_orders()

        # C1: Ensure all resources are associated with an order to increase throughput
        self._solver.Add(
            self._solver.Sum(self._var_x[v, o] for v in vehicles for o in orders) == min(len(vehicles), len(orders))
        )

        # C2: Each order must be assigned to at most one vehicle        
        for o in orders:
            self._solver.Add(
                self._solver.Sum(self._var_x[v, o] for v in vehicles) <= 1
            )

        # C3: Each vehicle must be assigned to at most one order        
        for v in vehicles:
            self._solver.Add(
                self._solver.Sum(self._var_x[v, o] for o in orders) <= 1
            )

        # C4: Number of vehicles dispatched to a location should not exceed its capacity
        for loc, loc_data in locations.items():
            expr_1 = [
                self._var_x[v, o]
                for v in vehicles
                for o in orders
                if orders[o]['origin'] == loc
            ]
            self._solver.Add(
                self._solver.Sum(expr_1) <= loc_data['capacity'] * self._capacity_violation_factor
            )

            expr_2 = [
                self._var_x[v, o]
                for v in vehicles
                for o in orders
                if orders[o]['dest'] == loc
            ]
            self._solver.Add(
                self._solver.Sum(expr_2) <= loc_data['capacity'] * self._capacity_violation_factor
            )

    def opt_ended(self):
        remaining_orders = self._data_center.get_remaining_orders()
        return len(remaining_orders) == 0

    @property
    def opt_obj(self):
        return self._opt_obj

    @property
    def opt_x(self):
        return self._opt_x

    @property
    def opt_results(self):
        return self._opt_results

In [167]:
data_center = VSDataCenter(data_file=file_path)

In [168]:
vs_solver = VSSolver(data_center=data_center)

In [169]:
while not vs_solver.opt_ended():
    vs_solver.optimize()
    for (v, o) in vs_solver.opt_x:
        print((v, o))

factor:  1.0
0
('SC001', 'CO_TFTU000029')
('SC002', 'CO_TFTU000229')
('SC003', 'CO_TFTU000209')
('SC004', 'CO_TFTU000109')
('SC005', 'CO_TFTU000157')
('SC006', 'CO_TFTU000271')
('SC007', 'CO_TFTU000115')
('SC008', 'CO_TFTU000207')
('SC009', 'CO_TFTU000062')
('SC010', 'CO_TFTU000094')
('SC011', 'CO_TFTU000111')
('SC012', 'CO_TFTU000120')
('SC013', 'CO_TFTU000045')
('SC014', 'CO_TFTU000262')
('SC015', 'CO_TFTU000113')
('SC016', 'CO_TFTU000089')
('SC017', 'CO_TFTU000066')
('SC018', 'CO_TFTU000070')
('SC019', 'CO_TFTU000142')
('SC020', 'CO_TFTU000090')
factor:  1.0
factor:  2.0
0
('SC001', 'CO_TFTU000096')
('SC002', 'CO_TFTU000052')
('SC003', 'CO_TFTU000110')
('SC004', 'CO_TFTU000254')
('SC005', 'CO_TFTU000277')
('SC006', 'CO_TFTU000267')
('SC007', 'CO_TFTU000139')
('SC008', 'CO_TFTU000284')
('SC009', 'CO_TFTU000246')
('SC010', 'CO_TFTU000283')
('SC011', 'CO_TFTU000244')
('SC012', 'CO_TFTU000248')
('SC013', 'CO_TFTU000080')
('SC014', 'CO_TFTU000195')
('SC015', 'CO_TFTU000151')
('SC016', 'C